# Training and Test Data

Augmented time series sub-sampled to every third timestep
The tail of each time-series series[:-prediction_length] is stored in validation set for validating model performance.

In [ ]:
from autogluon.common import space

chronos2_hyperparameters = {
    # https://auto.gluon.ai/stable/tutorials/timeseries/forecasting-model-zoo.html#pretrained-models
    "Chronos2": {
        "fine_tune": True,
        "fine_tune_mode": "lora",
        "fine_tune_lr": space.Real(1e-5, 1e-3, log=True),
        "fine_tune_steps": 3000,
        "fine_tune_batch_size": space.Categorical(32, 64, 128),
        "cross_learning": space.Bool(),
        "fine_tune_lora_config": {"r": 16, "lora_alpha": 32, "lora_dropout": 0.1},
    }
}

In [ ]:
from datetime import datetime
from functools import cache
import numpy as np
import pandas as pd
from fusiontimeseries.finetuning.preprocessing.utils import get_valid_flux_traces


@cache
def create_training_flux_dataframe() -> pd.DataFrame:
    training_flux_traces: dict[int, np.ndarray] = get_valid_flux_traces(
        full_subsampling=True
    )

    records = []
    for item_id, flux_trace in training_flux_traces.items():
        for t in range(flux_trace.shape[0]):
            records.append(
                {
                    "item_id": item_id,
                    "timestamp": pd.to_datetime(datetime(2000, 1, 1))
                    + pd.to_timedelta(t, unit="ms"),
                    "target": flux_trace[t],
                }
            )
    training_flux_df = pd.DataFrame(records)
    return training_flux_df


training_flux_df = create_training_flux_dataframe()
training_flux_df.head()

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame

training_data = TimeSeriesDataFrame.from_data_frame(
    training_flux_df,
    id_column="item_id",
    timestamp_column="timestamp",
)
training_data.size

In [ ]:
from autogluon.timeseries import TimeSeriesPredictor

predictor = TimeSeriesPredictor(
    prediction_length=64, target="target", eval_metric="RMSE", verbosity=4
)
predictor.fit(
    train_data=training_data,  # your training TimeSeriesDataFrame
    hyperparameters={**chronos2_hyperparameters},
    time_limit=3600,
    enable_ensemble=False,
    verbosity=4,
    hyperparameter_tune_kwargs={
        "num_trials": 8,
        "searcher": "bayes",
        "scheduler": "local",
    },
)

In [ ]:
from autogluon.timeseries import TimeSeriesPredictor

raw_predictor_path = input("Enter path to save the predictor: ")
predictor = TimeSeriesPredictor.load(raw_predictor_path)

In [ ]:
# Complete evaluation with automatic saving of all results
from fusiontimeseries.finetuning.evaluation_utils import (
    FinetuningConfig,
    run_complete_evaluation,
)

# Create configuration object
finetuning_config = FinetuningConfig(
    model_name="Chronos2\\T2",  # TODO add manually
    prediction_length=predictor.prediction_length,
    target="target",
    eval_metric="RMSE",
    hyperparameters={
        "fine_tune": True,
        "fine_tune_mode": "lora",
        "fine_tune_lr": 0.00048812550121497074,
        "fine_tune_steps": 3000,
        "fine_tune_batch_size": 64,
        "cross_learning": 0,
        "fine_tune_lora_config": {"r": 16, "lora_alpha": 32, "lora_dropout": 0.1},
    },  # TODO add manually
    time_limit=3600,
    start_context_length=80,
    relevant_tail_length=80,
)

# Run complete evaluation (forecasts, evaluates, plots, and saves everything)
results, json_path, plots_dir = run_complete_evaluation(
    predictor=predictor,
    config=finetuning_config,
    training_data_size=training_data.num_items,
    predictor_path=predictor.path if hasattr(predictor, "path") else None,
)

print("\n✅ All results saved!")
print(f"📊 Plots directory: {plots_dir}")
print(f"📄 JSON results: {json_path}")